# Bradings of T-branes

We want to reproduce the answer from Peng's notes

In [ ]:
from klrw.framed_dynkin import FramedDynkinDiagram_with_dimensions
from klrw.klrw_algebra import KLRWAlgebra
from klrw.perfect_complex import (
    KLRWPerfectComplex,
    KLRWProjectiveModule,
    KLRWHomOfPerfectComplexes,
)

## Setup

Defining the framed quiver

In [ ]:
DD = FramedDynkinDiagram_with_dimensions(CartanType(["A", 1]))
DD.inject_nodes(scope=globals())

DD[V1] = 2
DD[W1] = 2

print("Framed quiver:")
print(DD)

Defining the KLRW algebra

In [ ]:
R = QQ
KLRW = KLRWAlgebra(
    R,
    DD,
    edge_scaling=False,
    vertex_scaling=False,
    default_vertex_parameter=R.one(),
    default_edge_parameter=R.one(),
    warnings=True,
)

grading_group = KLRW.grading_group

Now it's time to write Peng's candidate for a braded T-brane.  
We introduce the signs to make it square to zero.

In [ ]:
# From Peng's notes
# The intersections with the horizontal lines are
# A1A2 | B2B1 | C1C2
# The vertical line separations are punctures

projectives = {
    0: [KLRWProjectiveModule(KLRW.state((W1, V1, V1, W1)), 0, grading_group)],  # B1B2
    1: [
        KLRWProjectiveModule(KLRW.state((V1, W1, V1, W1)), 1, grading_group),  # A1B2
        KLRWProjectiveModule(KLRW.state((W1, V1, W1, V1)), 1, grading_group),  # B1C2
        KLRWProjectiveModule(KLRW.state((W1, V1, W1, V1)), -1, grading_group),  # C1B2
        KLRWProjectiveModule(KLRW.state((V1, W1, V1, W1)), -1, grading_group),  # B1A2
    ],
    2: [
        KLRWProjectiveModule(KLRW.state((V1, V1, W1, W1)), 0, grading_group),  # A1A2
        KLRWProjectiveModule(KLRW.state((V1, W1, W1, V1)), 0, grading_group),  # A1C2
        KLRWProjectiveModule(KLRW.state((W1, W1, V1, V1)), 0, grading_group),  # C1C2
        KLRWProjectiveModule(KLRW.state((V1, W1, W1, V1)), -2, grading_group),  # C1A2
    ],
}


differentials = {}
differentials[1] = matrix(
    KLRW,
    len(projectives[1]),
    len(projectives[0]),
    sparse=True,
)

differentials[1][0, 0] = KLRW.KLRWmonomial(
    projectives[0][0].state, word=(1, 2)
)  # A1B2-> B1B2
differentials[1][1, 0] = KLRW.KLRWmonomial(
    projectives[0][0].state, word=(3, 2)
)  # B1C2-> B1B2
differentials[1][2, 0] = KLRW.KLRWmonomial(
    projectives[0][0].state, word=(3,)
)  # C1B2-> B1B2
differentials[1][3, 0] = KLRW.KLRWmonomial(
    projectives[0][0].state, word=(1,)
)  # B1A2-> B1B2


differentials[2] = matrix(
    KLRW,
    len(projectives[2]),
    len(projectives[1]),
    sparse=True,
)

differentials[2][0, 0] = KLRW.KLRWmonomial(
    projectives[1][0].state, word=(2,)
)  # A1A2 -> A1B2
differentials[2][0, 3] = -KLRW.KLRWmonomial(
    projectives[1][3].state, word=(1, 2)
)  # A1A2 -> B1A2
differentials[2][1, 0] = KLRW.KLRWmonomial(
    projectives[1][0].state, word=(3,)
)  # A1C2 -> A1B2
differentials[2][1, 1] = -KLRW.KLRWmonomial(
    projectives[1][1].state, word=(1,)
)  # A1C2 -> B1C2
differentials[2][2, 1] = KLRW.KLRWmonomial(
    projectives[1][1].state, word=(2,)
)  # C1C2 -> B1C2
differentials[2][2, 2] = -KLRW.KLRWmonomial(
    projectives[1][2].state, word=(3, 2)
)  # C1C2 -> C1B2
differentials[2][3, 2] = KLRW.KLRWmonomial(
    projectives[1][2].state, word=(1,)
)  # C1A2 -> C1B2
differentials[2][3, 3] = -KLRW.KLRWmonomial(
    projectives[1][3].state, word=(3,)
)  # C1A2 -> B1A2

T_twisted = KLRWPerfectComplex(
    KLRW_algebra=KLRW,
    differentials=differentials,
    projectives=projectives,
    check=True,
)

T_twisted

`check=True` option made sure that the differential squares to zero and that degrees are correct.  
So, it's a good complex!

## Exts

We will need a computation of exts:

In [ ]:
def find_exts(domain, codomain, rang=None, max_eq_deg = 10, min_eq_deg=-10):
    exts ={}
    ext_dims = {}

    if rang is None:
        from itertools import product
        max_hom_deg = max(-deg1 + deg2 for deg1 in domain.projectives for deg2 in codomain.projectives)
        min_hom_deg = min(-deg1 + deg2 for deg1 in domain.projectives for deg2 in codomain.projectives)
        rang = product(range(min_hom_deg, max_hom_deg+1), range(min_eq_deg, max_eq_deg+1))

    for shift, i in rang:
        print("Working on {} {}".format(shift, i))
        rhom = KLRWHomOfPerfectComplexes(domain, codomain[shift,i])
        ext = rhom.ext(0)
        dim = len(ext.basis())
        if dim:
            exts[shift, i] = ext
            ext_dims[shift, i] = dim

    return exts, ext_dims

We expect Exts from braided T to itself to be only in Maslov degree 0 [projectives!] and in equvariant degrees -2 [simple crossing] and higher.  
Let's compute the first several.

In [ ]:
exts, ext_dims = find_exts(T_twisted, T_twisted, rang=((0,0),(0,-1),(0,-2)))
ext_dims

As expected, we got 1 ext in equivariant degree -2 [the simple crossing] and no exts in equivariant degree -1.  
We can also see that the three exts in equivariant degree 0 match 3 braids between untwisted T-branes:

In [ ]:
state = KLRW.state((W1, V1, V1, W1))
KLRW[state: state: grading_group.zero()].basis()

Now it's time to look at the ext of equivariant degree -2.  
It is the image of the simple crossing.

In [ ]:
crossing_ext = exts[0,-2].basis()[0]
crossing_ext

It might be hard to read from this matrix, let's summarize it in Peng's notation for intersection points.  

We write maps as  
state -> state *coefficient  
Braids are often clear from the context. If they are not, look at the matrix.


In (Maslov) degree 1:

* C1B2 -> A1B2 *1
* C1B2- > B1C2 *(-1)

In (Maslov) degree 2:

* C1A2 -> A1A2 *1
* C1A2 -> A1C2 *1
* C1C2 -> C1C2 *1


## Matching

Our answer doesn't look similar to Peng's.  
No worries, there are chain homotopies, so our answers can still be chain homotopic.  

Let's look at all the chain homotopies.

In [ ]:
relations = exts[0,-2].relations()
from pprint import pprint
for x in relations:
    print("==============================")
    pprint(x)

After some meditation we find the desired correction.

In [ ]:
modified_crossing_ext =  crossing_ext - relations[0] - relations[1] -relations[3]
modified_crossing_ext

This is

In (Maslov) degree 0:

* B1B2 -> B1B2 *(-1)

In (Maslov) degree 1:

* B1A2 -> A1B2 *(-1)
* C1B2 -> B1C2 *(-1)

In (Maslov) degree 2:

* A1A2 -> A1A2 *1
* C1A2 -> A1C2 *1
* C1C2 -> C1C2 *1

This is Peng's answer!

Hooray!

## Square

Now we check if the elements squares to zero.  
For technical reason the second one has to be shifted in equivariant degree

In [ ]:
def shift_chain_map(chain_map, shift):
    domain = chain_map.parent().domain
    codomain = chain_map.parent().codomain
    rhom = KLRWHomOfPerfectComplexes(domain[shift], codomain[shift])
    ext = rhom.ext(0)
    hom = ext.hom_of_graded

    return hom._from_dict(
        {
            stype: entry
            for stype, entry in chain_map
        }
    )

In [ ]:
crossing_ext_shifted = shift_chain_map(crossing_ext, shift = (0, -2))
crossing_ext_shifted*crossing_ext

Zero, as expected!

## Dots

In [ ]:
from klrw.perfect_complex import SummandType

KLRW.base().inject_variables()

x1_dict = {
    SummandType(domain_grading=hom_deg, domain_index=i, codomain_index=i):
    KLRW.KLRWmonomial(projs[i].state)
    for hom_deg, projs in T_twisted.projectives.items()
    for i in range(len(projs))
}

# terms proportional to the identity
stype = SummandType(domain_grading=0, domain_index=0, codomain_index=0)
x1_dict[stype] = x_1_2 * x1_dict[stype] # B1B2
stype = SummandType(domain_grading=1, domain_index=0, codomain_index=0)
x1_dict[stype] = x_1_1 * x1_dict[stype]  # A1B2
stype = SummandType(domain_grading=1, domain_index=1, codomain_index=1)
x1_dict[stype] = x_1_1* x1_dict[stype] # B1C2
stype = SummandType(domain_grading=1, domain_index=2, codomain_index=2)
x1_dict[stype] = x_1_2 * x1_dict[stype]  # C1B2
stype = SummandType(domain_grading=1, domain_index=3, codomain_index=3)
x1_dict[stype] = x_1_2 * x1_dict[stype] # B1A2
stype = SummandType(domain_grading=2, domain_index=0, codomain_index=0)
x1_dict[stype] = x_1_1 * x1_dict[stype]  # A1A2
stype = SummandType(domain_grading=2, domain_index=1, codomain_index=1)
x1_dict[stype] = x_1_1 * x1_dict[stype]  # A1C2
stype = SummandType(domain_grading=2, domain_index=2, codomain_index=2)
x1_dict[stype] = x_1_1 * x1_dict[stype]  # C1C2
stype = SummandType(domain_grading=2, domain_index=3, codomain_index=3)
x1_dict[stype] = x_1_2 * x1_dict[stype]  # C1A2


# special terms
stype = SummandType(domain_grading=1, domain_index=0, codomain_index=3)
x1_dict[stype] = - KLRW.KLRWmonomial(T_twisted.projectives[1][0].state) # A1B2 -> B1A2
stype = SummandType(domain_grading=1, domain_index=1, codomain_index=2)
x1_dict[stype] = - KLRW.KLRWmonomial(T_twisted.projectives[1][1].state) # B1C2 -> C1B2
stype = SummandType(domain_grading=2, domain_index=1, codomain_index=3)
x1_dict[stype] = KLRW.KLRWmonomial(T_twisted.projectives[2][1].state) # A1C2 -> C1A2

hom = KLRWHomOfPerfectComplexes(T_twisted, T_twisted[0,2]).ext(0).hom_of_graded
x1 = hom._from_dict(
    x1_dict,
    coerce=False,
)
x1

We want to check if it's indeed a chain complex. To do this, we try to make a cone.

In [ ]:
x1.cone()

Since there is no error in making the cone, it's indeed a chain map!

Now let's do the other dot.

In [ ]:
x2_dict = {
    SummandType(domain_grading=hom_deg, domain_index=i, codomain_index=i):
    KLRW.KLRWmonomial(projs[i].state)
    for hom_deg, projs in T_twisted.projectives.items()
    for i in range(len(projs))
}

# terms proportional to the identity
stype = SummandType(domain_grading=0, domain_index=0, codomain_index=0)
x2_dict[stype] = x_1_1 * x2_dict[stype] # B1B2
stype = SummandType(domain_grading=1, domain_index=0, codomain_index=0)
x2_dict[stype] = x_1_2 * x2_dict[stype]  # A1B2
stype = SummandType(domain_grading=1, domain_index=1, codomain_index=1)
x2_dict[stype] = x_1_2* x2_dict[stype] # B1C2
stype = SummandType(domain_grading=1, domain_index=2, codomain_index=2)
x2_dict[stype] = x_1_1 * x2_dict[stype]  # C1B2
stype = SummandType(domain_grading=1, domain_index=3, codomain_index=3)
x2_dict[stype] = x_1_1 * x2_dict[stype] # B1A2
stype = SummandType(domain_grading=2, domain_index=0, codomain_index=0)
x2_dict[stype] = x_1_2 * x2_dict[stype]  # A1A2
stype = SummandType(domain_grading=2, domain_index=1, codomain_index=1)
x2_dict[stype] = x_1_2 * x2_dict[stype]  # A1C2
stype = SummandType(domain_grading=2, domain_index=2, codomain_index=2)
x2_dict[stype] = x_1_2 * x2_dict[stype]  # C1C2
stype = SummandType(domain_grading=2, domain_index=3, codomain_index=3)
x2_dict[stype] = x_1_1 * x2_dict[stype]  # C1A2


# special terms
stype = SummandType(domain_grading=1, domain_index=0, codomain_index=3)
x2_dict[stype] = KLRW.KLRWmonomial(T_twisted.projectives[1][0].state) # A1B2 -> B1A2
stype = SummandType(domain_grading=1, domain_index=1, codomain_index=2)
x2_dict[stype] = KLRW.KLRWmonomial(T_twisted.projectives[1][1].state) # B1C2 -> C1B2
stype = SummandType(domain_grading=2, domain_index=1, codomain_index=3)
x2_dict[stype] = - KLRW.KLRWmonomial(T_twisted.projectives[2][1].state) # A1C2 -> C1A2

hom = KLRWHomOfPerfectComplexes(T_twisted, T_twisted[0,2]).ext(0).hom_of_graded
x2 = hom._from_dict(
    x2_dict,
    coerce=False,
)
x2

Again, checking if it's indeed a chain complex.

In [ ]:
x2.cone()

Time to check the relation

In KLRW it looks this way:

In [ ]:
x_1_ = x_1_1*KLRW.KLRWmonomial(state=KLRW.state((W1,V1,V1,W1)))
x_2_ = x_1_2*KLRW.KLRWmonomial(state=KLRW.state((W1,V1,V1,W1)))
crossing = KLRW.KLRWmonomial(state=KLRW.state((W1,V1,V1,W1)), word = (2,))
x_1_, x_2_, crossing

The first relation is:

In [ ]:
print(x_1_, "*", crossing)
print(" - ")
print(crossing, "*", x_2_)
print(" = ")
print(x_1_ * crossing - crossing * x_2_)

and the second one

In [ ]:
print(x_2_, "*", crossing)
print(" - ")
print(crossing, "*", x_1_)
print(" = ")
print(x_2_ * crossing - crossing * x_1_)

Let's check the relation now

In [ ]:
lhs = shift_chain_map(x1, shift=(0,-2)) * crossing_ext - shift_chain_map(crossing_ext, shift=(0,2)) * x2

rhom = KLRWHomOfPerfectComplexes(T_twisted, T_twisted)
ext = rhom.ext(0)
rhs = ext.hom_of_graded.one()

ext.reduce(lhs - rhs).is_zero()

In [ ]:
lhs = shift_chain_map(x2, shift=(0,-2)) * crossing_ext - shift_chain_map(crossing_ext, shift=(0,2)) * x1

rhs = - ext.hom_of_graded.one()

ext.reduce(lhs - rhs).is_zero()

Both relations work!